## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-01-06-25 +0000,nypost,Connecticut animal activists seeking jerks who...,https://nypost.com/2025/09/12/us-news/connecti...
1,2025-09-13-01-06-20 +0000,startribune,Live: Scores and updates from Week 3 football ...,https://www.startribune.com/live-scores-and-up...
2,2025-09-13-01-02-54 +0000,wapo,Tyler Robinson had not voted in any elections ...,https://www.washingtonpost.com
3,2025-09-13-01-02-54 +0000,wapo,Live updates: Charlie Kirk shooting suspect in...,https://www.washingtonpost.com/nation/2025/09/...
4,2025-09-13-01-01-30 +0000,nyt,"Who Is Sushila Karki, the New Leader of Nepal?",https://www.nytimes.com/2025/09/10/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,kirk,50
28,charlie,43
183,trump,23
30,shooting,16
31,suspect,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
265,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,168
9,2025-09-13-00-57-55 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,147
87,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,147
3,2025-09-13-01-02-54 +0000,wapo,Live updates: Charlie Kirk shooting suspect in...,https://www.washingtonpost.com/nation/2025/09/...,146
36,2025-09-12-23-09-42 +0000,nyt,"What We Know About Tyler Robinson, Suspect in ...",https://www.nytimes.com/2025/09/12/us/tyler-ro...,142


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
265,168,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
186,64,2025-09-12-14-46-32 +0000,nypost,Trump predicts ‘my little Communist’ Zohran Ma...,https://nypost.com/2025/09/12/us-news/trump-pr...
274,54,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
87,54,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...
282,42,2025-09-12-08-00-00 +0000,cbc,"Israel's attack on Qatar was an escalation, bu...",https://www.cbc.ca/news/world/israel-qatar-inv...
55,41,2025-09-12-22-00-15 +0000,nypost,Utah Gov. Spencer Cox tells people to ‘touch g...,https://nypost.com/2025/09/12/us-news/utah-gov...
78,36,2025-09-12-21-08-17 +0000,missionlocal,Developer pulls out of S.F. public housing whe...,https://missionlocal.org/2025/09/future-of-s-f...
172,33,2025-09-12-15-47-11 +0000,nypost,Ex-White House press secretary Karine Jean-Pie...,https://nypost.com/2025/09/12/us-news/ex-white...
77,32,2025-09-12-21-08-55 +0000,nypost,Ukraine strikes key Russian oil terminal in la...,https://nypost.com/2025/09/12/world-news/ukrai...
155,32,2025-09-12-17-08-00 +0000,bbc,Anti-apartheid hero's new inquest should lead ...,https://www.bbc.com/news/articles/c931n9eelpeo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
